# 3.3 线性回归的简单实现

在上一节中，除了张量和反向传播，训练所需要的其他工具（数据迭代器、模型、损失函数、优化器）我们都自己动手实践了一遍。实际上，成熟的深度学习框架已经将这些collections实现好了。本节将介绍如何使用pytorch简洁地实现上一节的线性回归模型。

## 3.3.1 生成数据集

In [1]:
import numpy as np
import torch

from torch.utils import data
from d2l import torch as d2l

In [5]:
true_w = torch.tensor([2, -3.4])
true_b = torch.tensor(4.2)

features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 3.3.2 读取数据集

在这里，我们直接调用框架中现有的API来读取数据：

In [33]:
def load_array(data_arrays, batch_size, is_train=True): #@save
    """pytorch数据迭代器"""
    
    dataset = data.TensorDataset(*data_arrays)
    
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [84]:
batch_size = 2
data_iter = load_array((features, labels), batch_size)

对data_iter使用内置函数iter()，可以返回一个可迭代的对象，并使用内置函数next()获取其第一个项：

In [37]:
next(iter(data_iter))

[tensor([[ 1.4907, -0.7659],
         [ 1.9876,  0.3372],
         [-0.7485,  0.1689],
         [-0.1056, -0.6278]]),
 tensor([[9.7773],
         [7.0311],
         [2.1121],
         [6.1386]])]

## 3.3.3 定义模型

对于标准深度学习模型的构建，可以用框架预定义好的模块，这使得我们只需要关注使用哪些模块来构造模型，而不必关注模块的具体实现。

首先，我们定义一个Sequential类的实例net，Sequential类可以将多个模块级联在一起。当给定输入数据时，该实例可以将数据传入第一层，并将第一层的输出作为第二层的输入，以此类推。

即使模型只有一层（一个模块），我们也会使用Sequential以符合“标准的流水线”。

In [39]:
import torch.nn as nn

net = nn.Sequential(nn.Linear(2, 1))

## 3.3.4 初始化模型参数

在使用net之前，还需要初始化模型参数。深度学习框架通常有预定义的方法来初始化参数。在这里，我们指定每个权重参数应该从均值为0、标准差为1的正态分布中随机抽样，偏置参数初始化为0。

In [104]:
# net[0]表示Sequential类实例net中的第一层
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 3.3.5 定义损失函数

这里使用均方误差MSELoss类，默认返回所有样本的平均损失。

In [43]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

Pytorch在optim模块中实现了许多优化算法。当我们创建一个SGD实例时，要指定优化的参数（通过net.parameters()）以及所需的超参数。

In [85]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

## 3.3.7 训练

在上面，我们使用Pytorch框架简洁地实现了训练所需的基本组件，下面的训练过程和我们从零开始实现时的非常相似：

In [112]:
net[0].weight.data.normal_(0, 1)
net[0].bias.data.fill_(0)

# 查看参数初始化情况
print(list(net.parameters()))
      
epochs = 10

for epoch in range(epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

[Parameter containing:
tensor([[-0.0041, -0.2703]], requires_grad=True), Parameter containing:
tensor([0.], requires_grad=True)]
epoch 1, loss 4.215328
epoch 2, loss 0.567284
epoch 3, loss 0.077122
epoch 4, loss 0.010640
epoch 5, loss 0.001560
epoch 6, loss 0.000307
epoch 7, loss 0.000133
epoch 8, loss 0.000108
epoch 9, loss 0.000105
epoch 10, loss 0.000104


比较训练得到的参数与真实参数的差异：

In [110]:
w = net[0].weight.data
b = net[0].bias.data

print('w Loss:', true_w - w.reshape(true_w.shape))
print('b Loss:', true_b - b.reshape(true_b.shape))

w Loss: tensor([-0.0001, -0.0003])
b Loss: tensor(-0.0002)
